# Comparing degenerations of Spinor varieties

*Authors: George Balla, Daniel Corey, Igor Makhlin, and Victoria Schleis*

In this notebook accompanying our paper "Regular subdivisions, bounds on initial ideals, and categorical limits", we compute degenerations of Spinor varieties. We show that for Spinor varieties of Type $D_4$ and $D_5$, we have that $in_w(I) = I_w$, where $I_w$ is the lower bound as defined in our paper. 

We refer to `Computing_I_w_and_Omega.ipynb` for a thorough explanation and documentation of the lower bound ideal $I_w$ and the computation of the fan $\Omega(I) = \{w \in \mathbb{R}^n | in_w(I) = I_w\}$, whereas this notebook only covers the extended example computation for Spinor varieties. 


Our code uses the package Oscar in version 1.6.1
    

In [2]:
using Oscar;
using Combinatorics;
pm = Polymake;

include("computing_Iw_Omega.jl")
include("gfan_functions.jl")

[ Info: Precompiling Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13] (cache misses: wrong dep version loaded (2))
/home/victoria/.julia/juliaup/julia-1.11.5+0.x64.linux.gnu/bin/julia: symbol lookup error: /home/victoria/.julia/artifacts/02903f8b1c5d24ff1b9ec785fc95b09933a40097/lib/libsingular_julia.so: undefined symbol: jl_arrayset, version JL_LIBJULIA_1.11


LoadError: Failed to precompile Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13] to "/home/victoria/.julia/compiled/v1.11/Oscar/jl_oExorb".



## Spinor varieties


The space of totally isotropic subspaces of a $2n$-dimensional vector space equipped with a symmetric, nondegenerate form of Witt index $n$ is an algebraic variety which consists of two irreducible components that are projectively equivalent to each other. The _Spinor variety_ $\mathbb{S}_{n}$ is one of these irreducible components. It is a projective variety via the Wick embedding $J_n\subset S$ where $S$ is a polynomial ring with a generator for each subset of $[1,n]$ that has the same number of elements as $n$ modulo $2$. In [Cor], it is shown that $(J_n^{\circ})_w = in_{w}(J_n)^{\circ}$ for $n\leq 5$ and $w\in \mathrm{Trop}(J_n)$. Here, we demonstrate the equality $(J_n)_w = in_{w}(J_n)$ for $n\leq 5$ and $w\in \mathrm{Trop}(J_n)$.

---

**Function:** `spinor_variety_D(n)`

*Description*: For an integer `n` computes the ideal generating the Spinor variety of type $D_n$.

In [3]:
function spinor_variety_D(n)
    
l(i, I) = length([j for j in I if j < i])
Le = vcat([reverse.(sort(reverse.(subsets(n+1, k)))) for k in 0:2:n+1]...)
Lo = vcat([subsets(n+1, k) for k in 1:2:n+1]...)
RDe, xDe = graded_polynomial_ring(QQ, :xDe => Le)
xDe = Dict(k => v for (k, v) in zip(Le, xDe))

relsDe = []
for (I, J) in subsets(Lo, 2)
    SD = symdiff(I, J)
    if length(SD) >= 4
        push!(relsDe, sum([(-1)^(l(i, I) + l(i, J))*xDe[sort(symdiff(I, i))]*xDe[sort(symdiff(J, i))] for i in SD]))
    end
end
return ideal(relsDe)
end

schubert_variety (generic function with 2 methods)

*Example*: We compute ideals of the Spinor varieties of type $D_4$ and $D_5.

In [ ]:
I = spinor_variety_D(4)

J = spinor_variety_D(5);

## The case $n = 4 $

In this section we focus on the Spinor variety associated to $D_4$, whose ideal `I` we computed in the previous section. 

We briefly recall the definitions of $I_w$ and $\Omega(I)$. Let $\Delta = \mathcal{A}(I)$ be the point configuration of $I$. A vector $\mathsf{w}\in \mathbb{R}^{n}$ induces a regular subdivision $\mathrm{subd}_{\mathsf{w}}(\Delta)$ of $\Delta$. We can form the ideals 

$$\{\tilde{I}_{B} \, : \, B \text{ is a maximal cell of } \mathrm{subd}_{\mathsf{w}}(\Delta)\}$$

and the ideal they generate is

$$I_{\mathsf{w}} = \sum_{B \text{ maximal}} \tilde{I}_{B}. $$

For an ideal $I$ with point configuration $\Delta$, we define $\Omega(I)$ by

$$ \Omega(I) = \{\mathsf{w} \in \mathbb{R}^{n} \, : \, \mathrm{in}_{\mathsf{w}} I = I_{\mathsf{w}} \}. $$

This is a subfan of the secondary fan $\mathrm{Sec}(\Delta)$.

---

To show that $in_w(I) = I_w$ for all weight vectors $w$, we compute $\mathrm{Trop}(I)$ using `tropical_variety`. For each maximal cone in  $\mathrm{Trop}(I)$, we obtain a weight $w$ in the relative interior, and compare the initial degeneration $in_w(I)$ (computed via `initial`) to the ideal $I_w$ we compute using `ideal_w` as documented in `Computing_I_w_and_Omega.ipynb`. In the process, we count whenever equality is achieved. We then compare this number to the number of maximal cells of $\mathrm{Trop}(I)$, and obtain equality.
 

In [4]:
RR, y = polynomial_ring(QQ, length(gens(base_ring(I)), :y);
g = hom(base_ring(Ifour), RR, gens(RR));
II = g(I);
TV = tropical_variety(II)[1];
println(length(maximal_polyhedra(TV)))

c = 0;
inIs = Set([]);
for C in maximal_polyhedra(TV)
    w = sum(rays_modulo_lineality(C)[1])
    w = lcm(denominator.(w))*w
    
    inI = initial(I, tropical_semiring_map(QQ, max), w)
    push!(inIs, inI)
    
    c += ideal_w(Ifour, -w) == inI
end
println(c, "\n", length(inIs))

schubert_lineality_space (generic function with 1 method)

## The case $n = 5$ 

We now turn to the Spinor variety associated to $D_5$, given by the ideal `J` computed above. Similar to the previous section, we wish to show that $in_w(J) = J_w$ for all $w \in \mathrm{Trop}(J)$. On the whole, we proceed very similarly to the case $n=4$, however, as the computation of the tropical variety takes a while, we load it in pre-computed using the function `parse_tropical_file` which we provide in the supplementary file `gfan_functions.jl`. 

Afterwards, we proceed analogously to the above:  For each maximal cone in  $\mathrm{Trop}(J)$, we obtain a weight $w$ in the relative interior, and compare the initial degeneration $in_w(J)$ (computed via `initial`) to the ideal $J_w$ we compute using `ideal_w` as documented in `Computing_I_w_and_Omega.ipynb`. In the process, we count whenever equality is achieved. We then compare this number to the number of maximal cells of $\mathrm{Trop}(J)$, and obtain equality. Again, for the sake of running time, this notebook only performs the comparison for the last 1000 maximal cones in the tropical variety.

In [ ]:

rays, M = parse_tropical_file("n6trop.txt", true);

c = 0
inIs = Set([])
for i in nrows(M) - 1000 : nrows(M)
    R = [r for (j, r) in enumerate(rays) if M[i, j]]
    w = sum(R)
    inI = initial(J, tropical_semiring_map(QQ, max), w)
    
    Iw = ideal_w(J, -w)
    c += Iw == inI
    
    push!(inIs, inI)
end
c
